In [9]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import joblib

## Лабораторная работа 2

### Задание 1

In [10]:
def generate_dataset(num_samples):
    data = []
    for _ in range(num_samples):
        object1_state = [
            random.choice([0, 1]),
            random.choice(['Сфера', 'Куб', 'Пирамида']),  # Номинальный признак
            random.choice(['Вперед', 'Назад', 'Лево', 'Право']),  # Порядковый признак
            random.uniform(0, 360)  # Количественный признак
        ]
        
        # Состояние объекта 2
        object2_state = [
            random.choice([0, 1]),  # Бинарный признак
            random.choice(['Сфера', 'Куб', 'Пирамида']),  # Номинальный признак
            random.choice(['Вперед', 'Назад', 'Лево', 'Право']),  # Порядковый признак
            random.uniform(0, 360)  # Количественный признак
        ]
        
        collision = random.choice(['Да', 'Нет'])
        
        data.append(object1_state + object2_state + [collision])
    
    columns = [
        'Объект1_Бинарный', 'Объект1_Форма', 'Объект1_Направление', 'Объект1_Угол',
        'Объект2_Бинарный', 'Объект2_Форма', 'Объект2_Направление', 'Объект2_Угол',
        'Коллизия'
    ]
    return pd.DataFrame(data, columns=columns)

datasets = []
sample_ranges = [(30, 100), (100, 500), (500, 1000), (1000, 2000)]
feature_ranges = ['4-7', '8-10', '10+']

for sample_range in sample_ranges:
    for _ in range(3):
        num_samples = random.randint(*sample_range)
        dataset = generate_dataset(num_samples)
        datasets.append(dataset)

for i, dataset in enumerate(datasets):
    dataset.to_csv(f'dataset_v2_{i+1}.csv', index=False)

### Задание 2

In [11]:
df = datasets[0]

X = df.drop('Коллизия', axis=1)
y = df['Коллизия']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_features = ['Объект1_Угол', 'Объект2_Угол']
numeric_transformer = StandardScaler()

categorical_features = ['Объект1_Форма', 'Объект1_Направление', 'Объект2_Форма', 'Объект2_Направление']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

binary_features = ['Объект1_Бинарный', 'Объект2_Бинарный']
binary_transformer = 'passthrough'

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('bin', binary_transformer, binary_features)
    ])

models = {
    # Логистическая регрессия - быстрый и интерпретируемый алгоритм
    'Logistic Regression': LogisticRegression(max_iter=1000),
    # Решающее дерево - быстрый и не требует масштабирования данных
    'Decision Tree': DecisionTreeClassifier(),
    # Случайный лес - устойчив к переобучению, хорош для разнородных данных
    'Random Forest': RandomForestClassifier(),
    # SVM - хорош для сложных границ решений, но требует больше ресурсов
    'SVM': SVC(probability=True),
    # Наивный Байес - очень быстрый, хорошо работает с категориальными данными
    'Naive Bayes': GaussianNB()
}

results = {}
for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    
    joblib.dump(pipeline, f'{name.replace(" ", "_")}_v2_model.pkl')
    print(f'{name}: Accuracy = {accuracy:.4f}')

print("\nРезультаты точности моделей:")
for name, accuracy in results.items():
    print(f"{name}: {accuracy:.4f}")

Logistic Regression: Accuracy = 0.7143
Decision Tree: Accuracy = 0.8571
Random Forest: Accuracy = 0.5714
SVM: Accuracy = 0.5714
Naive Bayes: Accuracy = 0.7143

Результаты точности моделей:
Logistic Regression: 0.7143
Decision Tree: 0.8571
Random Forest: 0.5714
SVM: 0.5714
Naive Bayes: 0.7143


In [12]:
# Определение лучшей модели
best_model_name = max(results, key=results.get)
best_model_accuracy = results[best_model_name]
print(f"\nЛучшая модель: {best_model_name} с точностью {best_model_accuracy:.4f}")

# Пример использования сохраненной модели
loaded_model = joblib.load(f'{best_model_name.replace(" ", "_")}_v2_model.pkl')
sample_data = X_test.iloc[[0]]
prediction = loaded_model.predict(sample_data)
print(f"\nПример предсказания для строки:\n{sample_data}\nПредсказание: {prediction[0]}")
print(f"Реальное значение: {y_test.iloc[0]}")


Лучшая модель: Decision Tree с точностью 0.8571

Пример предсказания для строки:
    Объект1_Бинарный Объект1_Форма Объект1_Направление  Объект1_Угол  \
26                 0      Пирамида               Право     85.140066   

    Объект2_Бинарный Объект2_Форма Объект2_Направление  Объект2_Угол  
26                 1      Пирамида               Назад    198.461156  
Предсказание: Нет
Реальное значение: Нет
